In [1]:
import numpy as np
import numpy.matlib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.distance import pdist, squareform
import matplotlib.animation as animation
from scipy.stats import gaussian_kde as gaussian_kde
import matplotlib.cm as cm
import torch


class RBF(torch.nn.Module):
    def __init__(self, sigma=None):
        super(RBF, self).__init__()
        self.sigma = sigma

    def __call__(self, X, Y, sig=None):
        if sig:
            self.sigma = sig
        
        XX = X.matmul(X.t())
        XY = XX
        YY = XX
        
        dnorm2 = -2 * XY + XX.diag().unsqueeze(1) + YY.diag().unsqueeze(0)

        # Apply the median heuristic (PyTorch does not give true median)
        if self.sigma is None:
            np_dnorm2 = dnorm2.detach().cpu().numpy()
            h = np.median(np_dnorm2) / (2 * np.log(X.size(0) + 1))
            sigma = np.sqrt(h).item()
        else:
            sigma = self.sigma

        gamma = 1.0 / (1e-8 + 2 * sigma ** 2)
        
        K_XY = (- gamma * dnorm2).exp()
        
        dK_XY = torch.zeros(X.shape)

        for i in range(X.shape[0]):
            dK_XY[i] = K_XY[i].matmul(X[i] - X) * 4 * gamma
     
        return K_XY, dK_XY  
    
class scaled_hessian_RBF2(torch.nn.Module):
    def __init__(self, sigma=None):
        super(scaled_hessian_RBF2, self).__init__()
        self.sigma = sigma
        
    
    def __call__(self, X, Y, metric=None):
        if torch.is_tensor(metric):
            self.sigma = metric
        else:
            self.sigma = torch.eye(X.shape[1])
       
        def compute_script(X, Y, sigma):
            K_XY = torch.zeros(X.size(0), Y.size(0))
            dK_XY = torch.zeros(X.shape)
            for i in range(X.shape[0]):
                sign_diff = Y[i, :] - X
                Msd = torch.matmul(sign_diff, sigma)
                K_XY[i, :] = torch.exp( - 0.5 * torch.sum(sign_diff * Msd, 1))
                dK_XY[i] = K_XY[i, :].matmul(Msd) * 2
            return K_XY, dK_XY

        K_XY, dK_XY = compute_script(X, Y, self.sigma)   
        return K_XY, dK_XY  

# 
class SVN():
    """
    Based on: https://github.com/activatedgeek/svgd
    """
    def __init__(self, kernel, optimizer,x):
        self.kernel = kernel
        self.optimizer = optimizer
        #self.log_p = log_p
        self.H_diag = None
        self.nParticles = None
        self.dim = None
        self.stepsize = 1.
        self.maxmaxshiftold = torch.tensor(float('inf'))
        self.maxmaxshiftold_np = np.inf
        self.maxshift = np.zeros(self.nParticles)
        self.H = torch.zeros(x.size(0), x.size(1), x.size(1))
        self.H_block_loss = torch.zeros(x.size(0), x.size(1), x.size(1))
        self.H_loss = None

        
    def phi(self, x,minus_grad):
   
        if self.dim is None:
            self.dim = x.size(1)
        if self.nParticles is None:
            self.nParticles = x.size(0)
        
 
        M = torch.mean(self.GaussNewtonHessian(minus_grad), dim=0)
        
        k_xx, grad_k = self.kernel(x.detach(), x.detach(), M)
    
        phi = (k_xx.detach().matmul(minus_grad) + grad_k) / self.nParticles
        loss = minus_grad.sum()
        
        return -phi, -loss 
    


        
    def step(self, x, alpha,neg_grad):
        """    
        For LBFGS
        """     
        def closure():
            self.optimizer.zero_grad()
            x.grad, loss = self.phi(x,neg_grad)
            return loss
        
        loss = closure()
        options = {'closure': closure, 'current_loss': loss}
        self.optimizer.step(options)
        
    def GaussNewtonHessian(self, J):
        return 2 * J.reshape([self.nParticles, self.dim, 1]) * \
               J.reshape([self.nParticles, 1, self.dim])
      
    def resetParticles(self, x):
        x = torch.randn(self.nParticles, self.dim)
        return x

In [2]:
#particle initialization
import matplotlib.pyplot as plt
import numpy as np
# specify params
def scatter_particles_uniformally_for_new_env(particles_size,env_size=[50,50]):
    n = particles_size
    shape = np.array([env_size[0], env_size[1]])
    sensitivity = 0.8 # 0 means no movement, 1 means max distance is init_dist

    # compute grid shape based on number of points
    width_ratio = shape[1] / shape[0]
    num_y = np.int32(np.sqrt(n / width_ratio)) + 1
    num_x = np.int32(n / num_y) + 1

    
    x = -50+np.linspace(0., shape[1]-1+90, num_x, dtype=np.float32)

    y = np.linspace(0., shape[1]-1+35, num_x, dtype=np.float32)

    coords = np.stack(np.meshgrid(x, y), -1).reshape(-1,2)

    # compute spacing
    init_dist = np.min((x[1]-x[0], y[1]-y[0]))
    min_dist = init_dist * (1 - sensitivity)

    assert init_dist >= min_dist
 

    # perturb points
    max_movement = (init_dist - min_dist)/2
    noise = np.random.uniform(
        low=-max_movement,
        high=max_movement,
        size=(len(coords), 2))
    coords += noise
    if coords.shape[0]<=particles_size:#coord size is not always equal to particle size,so add numbers equal to difference
        a=coords.shape[0]
        b=particles_size-a

      
        coords=np.vstack([coords,noise[2:2+b,:]])
    if coords.shape[0]>=particles_size:#coord size is not always equal to particle size,so remove numbers equal to difference
        a=coords.shape[0]
        b=a-particles_size
    
      
        coords=coords[b:,:]

    return coords
coords=scatter_particles_uniformally_for_new_env(200,[10,10])
print(coords.shape)
#plt.figure(figsize=(10,10))
#plt.scatter(coords[:,0], coords[:,1], s=10)
#plt.show()

(200, 2)


In [3]:


import random
import numpy as np
import torch
import math
import time
import os

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
matplotlib.use("TkAgg") #to animate the plots

from numpy.random import *


import sys
sys.path.append('./')
import time

from LBFGS import FullBatchLBFGS, LBFGS

dtype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor

device = "cuda" if torch.cuda.is_available() else "cpu"



plt.close("all")




#https://stackoverflow.com/questions/52228411/compute-target-roll-pitch-and-yaw-angles-given-initial-and-target-positions-and        }    
    
def get_bearing(current_x,current_y,previous_x,previous_y):
  
    #D = P2 - P1 = (x2 - x1, y2 - y1, z2 - z1)
    #Y2 = atan2(Dy, Dx)
    
    yaw = math.atan2(current_y-previous_y, current_x-previous_x)
    #(NB can use P2 = atan2(Dz, sqrt(Dx^2 + Dy^2)) instead for more robustness near the spherical poles)
    return yaw

def normalize_angle(angle):
    """Normalizes the angle to the range [-PI, PI].

    :param angle the angle to normalize
    :return normalized angle
    """
    center = 0.0
    n_angle = angle - 2 * math.pi * math.floor((angle + math.pi - center) / (2 * math.pi))
    assert(-math.pi <= n_angle <= math.pi)

    return n_angle

#=========helping functions to process laser readings and motion detection    
def get_laser_files (b):

    laser_data= b['laser_data_xy_at_t']
    laser_data_endpoint=b['dist_theta_at_t']
    return laser_data,laser_data_endpoint

# get the indices of non zero numbers from two laser_x_y data or laser_end_point data
def get_difference_index(laser_array1,laser_array2,file="end_point_dist"):
    x=laser_array2-laser_array1
    if (file=="end_point_dist"):
        non_zero_indices=np.nonzero(x)#it is single dimension
    if (file=="laser_x_y"):
        non_zero_indices=np.nonzero(x[:,0])#it has x and y dim. so find non zero using any dim
    if (non_zero_indices[0].size==0):#if index array has zero element, meaning both consective scans are same
        print("no motion detected ")
    
    return non_zero_indices


#difference laser scan is different in two ways. every next scan is different from previous
#1) it has new location of moving object 2) it shows full range readings on the previous location 
#of moving opbject where previous scan had located object. 
#so we want to ignore the full range readings in this case.

# returns laser readings which are not full range and corresponding indices in difference scan (not in full scan)
def ignore_full_range_reading_in_difference_scan(laser_range,difference_scan_ranges,difference_scan):
    #get the indices where difference scan does not has full range readings
    #this will be true where we have not full range,else false.
   
    not_full_range_indices_true_values=difference_scan_ranges!=laser_range
    #then gather all true values
    not_full_range_indices = np.nonzero(not_full_range_indices_true_values)
    #now we have indices where we have non-full range values
    difference_scan_not_full_ranges = difference_scan[not_full_range_indices]
    difference_scan_not_full_ranges =np.reshape(np.mean(difference_scan_not_full_ranges,axis=0),(1,2))
    
    return not_full_range_indices, difference_scan_not_full_ranges
#===========================================================    
class ParticleFilter:
    
    def __init__(
    
        self,
        particle_count,
        initial_state=[0,0,0,0,0,0],#6d state vector [x,y,vx,vy,ax,ay]. N.B. accelerations are zero if using constant velocity motion model
        gaussian_std=[0.2,0.1,0.1,0.01,0.01,0.02],#6d std for states
        iteration=20,
        bearing = [0],
        std_bearing=0.1,
        
        plot_prog=0,
        plot_particles=0,
        plot_pred_part=0,#plot prediction particles
        lr=0.01,
        #laser_files_path = "toy2_setting2/",
        #ground_truth_poses ="toy2_setting2/_robot_poses.txt"
        
     ):  
        
        self.plot = plot_prog
        self.plot_particles=plot_particles
        self.plot_pred_part=plot_pred_part
      
        
        self.legend=1
        self.particle_count = particle_count
        
        self.previous_time=0
        self.time = 1
        self.iteration=iteration
       
       
        #previous state
        self.previous_disp_x = initial_state[0]
        self.previous_disp_y = initial_state[1]
        self.theta           = bearing[0]
        
        
        self.previous_vel_x = initial_state[2]
        self.previous_vel_y = initial_state[3]
        
        self.previous_acc_x = initial_state[4]
        self.previous_acc_y =initial_state[5]
        
        #current state. all updates will be fed into these variables
        self.current_disp_x=0
        self.current_disp_y=0
        self.current_vel_x=0
        self.current_vel_y=0
        self.current_acc_x=0
        self.current_acc_y=0
        
        #gaussian noise 
        self.std_dis_x=gaussian_std[0]
        self.std_dis_y=gaussian_std[1]
        
        self.std_vel_x=gaussian_std[2]
        self.std_vel_y=gaussian_std[3]
        
        self.std_acc_x=gaussian_std[4]
        self.std_acc_y=gaussian_std[5]
        
        self.std_bearing=std_bearing
        
       
       
    
        self.lr =lr
        
        
        self.device="cpu"
        self.optimizer="None"
       
        #directory where generated files live
        self.laser="toy2_setting2/"
        
        
        #torch version 
        self.x_tensor    = torch.empty(self.particle_count,1).normal_(mean=initial_state[0],
                                                             std=self.std_dis_x).type(dtype).to(device)
        self.y_tensor    = torch.empty(self.particle_count,1).normal_(mean=initial_state[1],
                                                            std=self.std_dis_y).type(dtype).to(device)
        self.theta_tensor= torch.empty(self.particle_count,1).normal_(mean=self.theta,      
                                                            std=self.std_bearing).type(dtype).to(device)
        
        self.vx_tensor   = torch.empty(self.particle_count,1).normal_(mean=initial_state[2],
                                                            std=self.std_vel_x).type(dtype).to(device)
        self.vy_tensor   = torch.empty(self.particle_count,1).normal_(mean=initial_state[3],
                                                            std=self.std_vel_y).type(dtype).to(device)
        
        self.ax_tensor   = torch.empty(self.particle_count,1).normal_(mean=initial_state[4],
                                                            std=self.std_acc_x).type(dtype).to(device)
        self.ay_tensor   = torch.empty(self.particle_count,1).normal_(mean=initial_state[5],
                                                            std=self.std_acc_y).type(dtype).to(device) 
        
        #update model related
        self.laser_position           = torch.zeros((1,2)).type(dtype).to(device)
        self.heading                  =0
        self.previous_laser_position  = torch.zeros((1,2)).type(dtype).to(device)#required to estimate velocity
        self.laser_vels               = torch.zeros((1,2)).type(dtype).to(device)
        self.previous_laser_vels      = torch.zeros((1,2)).type(dtype).to(device)
        self.laser_acc                = torch.zeros((1,2)).type(dtype).to(device)
        self.previous_laser_acc       = torch.zeros((1,2)).type(dtype).to(device)
       
    
        if self.plot:
            self.fig, self.ax = plt.subplots(figsize=(7,3))
        
       
        self.wframe=None
        self.wframe2=None #to avoid removing few intermediate frames of distributions
       
        
        self.predict_count=0
        self.stein_count=0
        self.laser_count=0
        self.predict_obstacle=0#flag will be set to 1 used once obstacle is detected to aid in ploting bi-modelilty
        self.obstacle_width=0
    
    def predict (self, 
                        
                        obstacle=0,      #when we have obstacle on the way,flag this to 1
                        delta_x=0,       #if obstacle is along y direction, get the x dim of obstacle
                                         #to split prediction on the sides of obstacle
                        delta_y=0):      #if obstacle is along x direction, get the y dim of obstacle 
                                         #to split prediction on top and bottom of obstacle
        
       
            
        #IF UPDATE MODEL IS BEING APPLIED, PREVIOUS STATE HAS TO BE MEAN PARTICLE LOCATION 
        self.previous_disp_x =torch.mean(self.x_tensor)
        self.previous_disp_y = torch.mean(self.y_tensor)

        self.previous_vel_x = torch.mean(self.vx_tensor)
        self.previous_vel_y = torch.mean(self.vy_tensor)

        
        self.current_disp_x =  (self.previous_vel_x*(self.time-self.previous_time) +
                                0.5*self.previous_acc_x*((self.time-self.previous_time)**2)) #s_x = ut + 0.5at^2\n",
        self.current_disp_y =  (self.previous_vel_y*(self.time-self.previous_time) + 
                                0.5*self.previous_acc_y*((self.time-self.previous_time)**2) )#s_y = ut + 0.5at^2 \n",
    
            
            
       
        #it will be constant as increment is always constant current=previous+const_delta
        self.current_theta = get_bearing((self.previous_disp_x+self.current_disp_x),
                                 (self.previous_disp_y+self.current_disp_y),
                                 self.previous_disp_x,
                                 self.previous_disp_y)
        
        
       
        self.current_vel_x= ((self.previous_disp_x+self.current_disp_x)- self.previous_disp_x )/(self.time-self.previous_time)
        self.current_vel_y= ((self.previous_disp_y+self.current_disp_y)-self.previous_disp_y )/(self.time-self.previous_time)
        
       
        #accelartion will be zero as velocity is constant
        self.current_acc_x= (self.current_vel_x- self.previous_vel_x )/(self.time-self.previous_time)
        self.current_acc_y= (self.current_vel_y- self.previous_vel_y )/(self.time-self.previous_time)
        
        rand_x= torch.empty(self.particle_count,1).normal_(mean=self.current_disp_x,std=self.std_dis_x)
        rand_y= torch.empty(self.particle_count,1).normal_(mean=self.current_disp_y,std=self.std_dis_y)

        
        if (obstacle==0):
            #without transformation
            self.x_tensor      = self.x_tensor + rand_x
            self.y_tensor      = self.y_tensor + rand_y
           
            
        else:#if we have obstacle in way at certain time stamp, predict bi-modal distribution
            #in case of obstacle along x, split the particle on the top and bottom of obstacle 
            #0.2 is added and subtracted to have some distance between motion and obstacle
            s=int(np.around(self.particle_count/2, decimals=0))
            rand_y1= torch.empty(self.particle_count,1).normal_(mean=self.current_disp_y+delta_y/2 + 0.2,std=self.std_dis_y)
            rand_y2= torch.empty(self.particle_count,1).normal_(mean=self.current_disp_y-delta_y/2 -0.2,std=self.std_dis_y)
            self.x_tensor      = self.x_tensor + rand_x
            #split particle half anf half to move on the top and bottom of obstacle
            self.y_tensor[:s,:]      = self.y_tensor[:s,:] + rand_y1[:s,:]
            self.y_tensor[s:,:]      = self.y_tensor[s:,:] + rand_y2[s:,:]
            
            self.predict_obstacle=1
            self.obstacle_width = delta_x
            
            
        self.theta_tensor  = self.theta_tensor + torch.empty(self.particle_count,1).normal_(
                                                                            mean=self.current_theta,
                                                                            std=self.std_bearing)
        self.vx_tensor = torch.empty(self.particle_count,1).normal_(
                                                                    mean=self.current_vel_x,
                                                                    std=self.std_vel_x)
        self.vy_tensor = torch.empty(self.particle_count,1).normal_(
                                                                    mean=self.current_vel_y,
                                                                    std=self.std_vel_y)

        self.ax_tensor = torch.empty(self.particle_count,1).normal_(
                                                                    mean=self.current_acc_x,
                                                                    std=self.std_acc_x)
        self.ay_tensor =  torch.empty(self.particle_count,1).normal_(
                                                                    mean=self.current_acc_y,
                                                                    std=self.std_acc_y)
        
                    
        if (self.plot_pred_part==1):#plot prediction particles?
            
            if ( self.predict_count==0):

                self.ax.scatter((self.x_tensor),(self.y_tensor),
                                         marker="o",facecolors='none', edgecolors='grey',
                                         label="Prediction particles")
                self.ax.legend() 

            else:

                self.wframe=self.ax.scatter((self.x_tensor),(self.y_tensor),
                                         marker="o",facecolors='none', edgecolors='grey')#

            
            
            plt.pause(0.01)
        
        if self.plot:           
            if (self.predict_obstacle==0):#when no obstacle
                if (self.predict_count==0):
                    self.ax.plot((torch.mean(self.x_tensor)),
                             (torch.mean(self.y_tensor)),
                             marker="*",color="red",label="Motion prediction")

                else:
                        self.ax.plot((torch.mean(self.x_tensor)),
                                     (torch.mean(self.y_tensor)),
                                     marker="*",color="red")            

            #if (self.predict_obstacle==1):#if there is obstacle 

            if ((self.predict_obstacle<self.obstacle_width)&(self.predict_obstacle!=0)):#with obstacle
                s=int(np.around(self.particle_count/2, decimals=0))
                if (self.predict_count==0):
                        self.ax.plot((torch.mean(self.x_tensor[:s,:])),
                                 (torch.mean(self.y_tensor[:s,:])),
                                 marker="*",color="red",label="Motion Prediction")

                        self.ax.plot((torch.mean(self.x_tensor[s:,:])),
                                 (torch.mean(self.y_tensor[s:,:])),
                                 marker="*",color="red")

                else:
                        self.ax.plot(torch.mean(self.x_tensor[:s,:]),
                                 (torch.mean(self.y_tensor[:s,:])),
                                     marker="*",color="red")
                        self.ax.plot(torch.mean(self.x_tensor[s:,:]),
                                 (torch.mean(self.y_tensor[s:,:])),
                                     marker="*",color="red")

                       
                self.predict_obstacle=self.predict_obstacle+1
                plt.pause(0.01)
            else: 
                if (self.predict_count==0):
                    self.ax.plot((torch.mean(self.x_tensor)),
                             (torch.mean(self.y_tensor)),
                             marker="*",color="red")

                else:
                    self.ax.plot((torch.mean(self.x_tensor)),
                                     (torch.mean(self.y_tensor)),
                                     marker="*",color="red")  

               

        
        else:#if not plotting at all
            self.predict_obstacle=self.predict_obstacle+1
            
       
        self.predict_count=self.predict_count+1
        return  
    
   
    
    def get_laser_pred(self,n,laser_range):#use laser scans and get the predicted location
        b=np.load(self.laser +'/toy2_setting2_frame_' + str(n-1) + '.npz')
        c=np.load(self.laser +'/toy2_setting2_frame_' + str(n) + '.npz')

        laser_x_y0,end0 =get_laser_files(b)
        laser_x_y1,end1 =get_laser_files(c)

        non_zero_indices=get_difference_index(laser_x_y0,laser_x_y1,"laser_x_y")

        current_scan_different_to_previous = laser_x_y1[non_zero_indices]
       
       
        #if only difference is that we have full range reading in difference scan, then report no motion detected
        if (current_scan_different_to_previous.shape[0]==1):
            if (end1[non_zero_indices]==laser_range):
                print("no motion either at n=", n)
                current_scan_different_to_previous=np.array([[[],[]]])#required to be compatible with plotting and sensing valid motion
                
        # if various beam difference, ignore full range beams as they appear just 
        #becuase of previous location of object being empty now and thus laser giving full range
        
        if (current_scan_different_to_previous.shape[0]>1):

               ss, current_scan_different_to_previous=ignore_full_range_reading_in_difference_scan(
                                                 laser_range,
                                                 end1[non_zero_indices],
                                                current_scan_different_to_previous)
               
               #angle will be the index where we have scan difference 
               
               
               
               
              
        
        if(np.isnan(current_scan_different_to_previous).all()==0):#if not nan
            
            if (self.laser_count==0):#for first iter attche label
                if self.plot:
                    self.ax.plot(current_scan_different_to_previous[:,0],
                                 current_scan_different_to_previous[:,1],marker="o",
                                 markersize='3',color="black",
                                 label="Ground-truth")
                    
                self.laser_count=self.laser_count+1
               
            else:
                if self.plot:
                    self.ax.plot(current_scan_different_to_previous[:,0],
                             current_scan_different_to_previous[:,1],marker="o",markersize='3'
                                 ,color="black")
               
            return torch.from_numpy(current_scan_different_to_previous).type(dtype)
        else:
            #nan numpy cannot be converted to
            return current_scan_different_to_previous
    


        
    def mean_gradients_for_lbfg(self,updated_tensor):
        gradient_cost = torch.zeros((self.particle_count, 7))
      
 
        gradient_cost[:,0] = (updated_tensor[:,0]-self.laser_position[:,0]).squeeze()
        gradient_cost[:,1] = (updated_tensor[:,1]-self.laser_position[:,1]).squeeze()
        gradient_cost[:,2] = (updated_tensor[:,2]-self.heading).squeeze()
    
        gradient_cost[:,3] = (updated_tensor[:,3]-self.laser_vels[:,0]).squeeze()
        gradient_cost[:,4] = (updated_tensor[:,4]-self.laser_vels[:,1]).squeeze()
        
        gradient_cost[:,5] = (updated_tensor[:,5]-self.laser_acc[:,0]).squeeze()
        gradient_cost[:,6] = (updated_tensor[:,6]-self.laser_acc[:,1]).squeeze()
        
        
        
        return gradient_cost   
   

    def state_estimate(self,
                       obstacle_on_way=0,#does the env has obstacle, if yes which time step
                       obstacle_time_step=0,
                       obstacle_width=0,
                       obstacle_height=0
                      
                      ):
        
        
       
        label="lbfg_stein"
        ground_truth_poses = "obstacle_same_path_later_change/_robot_poses.txt"
        self.laser="obstacle_same_path_later_change/"   
        laser_range=50
        coords=scatter_particles_uniformally_for_new_env(self.particle_count)
        self.x_tensor    = torch.from_numpy(coords[:self.particle_count,0]).resize(self.particle_count,1).type(dtype).to(device)
        self.y_tensor    = torch.from_numpy(coords[:self.particle_count,1]).resize(self.particle_count,1).type(dtype).to(device)


        
        self.ground_truth_poses = ground_truth_poses
        self.gt_x,self.gt_y,self.gt_theta=np.genfromtxt(self.ground_truth_poses,unpack=True)
        self.trajectory_len=len(self.gt_x)
            
        
        
        coun=0 
        traj=len(self.gt_x)
       
        for i in np.arange(1,traj):
            
            if (obstacle_on_way!=0):
                    if (self.time==obstacle_time_step):
                        self.predict(
                                     obstacle=obstacle_on_way,
                                     delta_x=obstacle_width,
                                     delta_y=obstacle_height)
                    else:
                        self.predict()
            else:

                self.predict()
           

            self.laser_position= self.get_laser_pred(i,laser_range)
           

            if (np.isnan(self.laser_position).all()):
                coun=coun+1
              
                self.previous_laser_position = np.reshape(torch.tensor([torch.mean(self.x_tensor),
                                                             torch.mean(self.y_tensor)]),(1,2))
                if self.plot:
                    self.ax.plot(self.gt_x[i],self.gt_y[i],
                                                 marker="x",color="black")

            
            else:
        
                self.heading = math.atan2(self.laser_position[:,1],self.laser_position[:,0])
               
                self.laser_vels = ((self.laser_position - self.previous_laser_position)/(
                                                            self.time-self.previous_time)).type(dtype) 
                
                self.laser_acc = (self.laser_vels - self.previous_laser_vels)/(self.time-self.previous_time)

                parameters = [self.x_tensor, self.y_tensor, self.theta_tensor,
                              self.vx_tensor, self.vy_tensor,
                             self.ax_tensor, self.ay_tensor]
                
                
               
                #===============================================================================        
    
                parameters2 = torch.transpose(torch.stack([self.x_tensor,
                                                           self.y_tensor, 
                                                           self.theta_tensor,
                                                           self.vx_tensor, 
                                                           self.vy_tensor,
                                                           self.ax_tensor, 
                                                           self.ay_tensor]).squeeze(),0,1)
               
                K = scaled_hessian_RBF2(sigma=None) 
                
                self.optimizer = FullBatchLBFGS(params=[parameters2], lr=self.lr, line_search='None')
                
             
                svn = SVN(kernel=K, optimizer=self.optimizer,x=parameters2)
                
               
                for n in range(self.iteration):
                   
                    mean_grads = self.mean_gradients_for_lbfg(parameters2)
                    svn.step(parameters2, alpha=1,neg_grad=-mean_grads)
                
                    self.x_tensor =parameters2[:,0].unsqueeze(dim=1)
                    self.y_tensor =parameters2[:,1].unsqueeze(dim=1)
                    self.theta_tensor =parameters2[:,2].unsqueeze(dim=1)
                    self.vx_tensor =parameters2[:,3]
                    self.vy_tensor =parameters2[:,4]
                    self.ax_tensor =parameters2[:,5]
                    self.ay_tensor =parameters2[:,6]
           
                #=================================================================================     
                if self.plot:    
                    if (self.plot_particles==0):#plot just mean

                        if (self.stein_count==0):

                            self.wframe=self.ax.plot(torch.mean(self.x_tensor),torch.mean(self.y_tensor),
                                                     marker="x",color="green",markersize='8',
                                                     label="SPF mean update")
                            plt.pause(0.001)

                        else:

                            self.wframe=self.ax.plot(torch.mean(self.x_tensor),torch.mean(self.y_tensor),
                                                     marker="x",color="green",markersize='8')#

                            plt.pause(0.01)


                    else:#plot patricles and mean
                        if ( self.stein_count==0):

                            self.ax.scatter((self.x_tensor),(self.y_tensor),
                                                     marker="o",facecolors='none', edgecolors='c',
                                                     label="SPF update particles")
                            self.ax.plot(torch.mean(self.x_tensor),torch.mean(self.y_tensor),
                                                     marker="x",color="green",markersize='8',
                                                     label="SPF mean update")


                            if self.legend==1:
                                self.ax.legend()
                            plt.pause(0.001)

                        else:

                            self.ax.scatter((self.x_tensor),(self.y_tensor),
                                                     marker="o",facecolors='none', edgecolors='c')#
                            #plot mean as well
                            self.ax.plot(torch.mean(self.x_tensor),torch.mean(self.y_tensor),
                                                     marker="x",color="green",markersize='8')

                            plt.pause(0.01)


                    self.stein_count= self.stein_count+1
                
                self.previous_laser_position = self.laser_position
            
        
            
           
            self.previous_time=self.time
            self.time=self.time+1
            
            if self.plot:
                self.ax.patch.set_edgecolor('black')
                self.ax.patch.set_linewidth('1.5')

                self.ax.set_xlim(-28,27)
                self.ax.set_ylim(8,16)

                
                self.ax.set_xlim(-50,50)
                self.ax.set_ylim(8,30)
                self.ax.tick_params(axis='x', labelsize='16' )
                self.ax.tick_params(axis='y', labelsize='16' )
                self.ax.legend()
            
        if self.plot:
            return self.ax,self.fig
        else:
            fig_p1, ax_p1 = plt.subplots(figsize=(1,1))
            return fig_p1, ax_p1
                





In [4]:
plt.close("all")
particle_size=50
iters  =35               # no of iters you want to run
lr=0.15

plot=1  #anyplot progress
plot_particles=0 
plot_prediction_particles=0 #plot prediction particles?



# following is fixed for the given environments
obstacle_on_way=4
obstacle_time_step=33
obstacle_width=10
obstacle_height=2  
    
import time
start_time = time.time()



initial_state=[0,0,1,0.02,0,0.0] #[x,y,vx,vy,ax,ay].for uniform velocity, accelaerations (ax and ay) are zeros



pf=ParticleFilter(  particle_count=50,
                    initial_state=initial_state,
      
                    gaussian_std=[0.2,0.1,0.1,0.01,0.01,0.02],

                    iteration=iters,
                    bearing = [0],
                    std_bearing=0.1,
       
    
                    plot_particles=plot_particles,
                    plot_pred_part=plot_prediction_particles,
                    plot_prog=plot,

                    lr=lr,#1
     )

t=time.time()
axi,f=pf.state_estimate(
                    obstacle_on_way=obstacle_on_way,
                    obstacle_time_step=obstacle_time_step,
                    obstacle_width=obstacle_width,
                    obstacle_height=obstacle_height
                   )


/home/fahira/anaconda3/lib/python3.7/site-packages/torch/tensor.py:474: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
/home/fahira/anaconda3/lib/python3.7/site-packages/torch/tensor.py:474: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
/home/fahira/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:86: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
/home/fahira/catkin_ws/src/icp_mcl_stein/python_particle_filter/code/LBFGS.py:236: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:1005.)
  p.data.add_(step_size, update[offset:offset + numel].view_as(p.data))
/home/fahira/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:86: Depre

no motion either at n= 3


/home/fahira/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:86: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
/home/fahira/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:86: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.


no motion either at n= 6
no motion detected 


/home/fahira/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:86: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
/home/fahira/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:86: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
/home/fahira/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:86: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
/home/fahira/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:86: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
/home/fahira/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:86: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
/home/fahira/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:86: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
/home/fahi

...

no motion either at n= 81
no motion either at n= 83


/home/fahira/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:86: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
/home/fahira/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:86: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.


no motion either at n= 85


/home/fahira/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:86: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
/home/fahira/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:86: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.


no motion either at n= 88
no motion detected 
